In [1]:
%pylab inline

from sklearn import tree
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from vocabulary import Vocabulary
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  #交叉验证
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

from time import time
import datetime
import pandas as pd
import numpy as np
import graphviz
import pickle
import warnings

warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


## Amino acid type

In [2]:
# 将序列残基对应词典转换成特征
data = pd.read_excel('/home/dldx/R-H/code/classification/ML/data/mafft_MSA_R_H.xlsx')

vocabulary = Vocabulary.get_vocabulary_from_sequences(data.Sequence.values)  #将蛋白质序列构造成词典

# 数据处理
input_seq_tensor = []
input_lable = []
for i in range(179):
    # 将序列残基进行字符编码
    seq_tensor = vocabulary.seq_to_tensor(data.Sequence[i])
    input_seq_tensor.append(np.array(seq_tensor))
    
input_seq_tensor = np.array(input_seq_tensor)
input_lable = np.array(data.Tier)

## Disorder score

In [8]:
data1 = pd.read_csv('/home/dldx/R-H/code/classification/ML/data/disorder/MSA_disorder_R_H.csv')

input_feature = pickle.load(open('/home/dldx/R-H/code/classification/ML/data/disorder/all_seq_disorder_R_H.pkl','rb'))
input_lable1 = np.array(data1.Tier)

## LR  Feature1   Amino acid type

In [15]:
# 设置超参数
param_dict = { "penalty": ["l1","l2"], "C": np.linspace(0.05,1,19), "random_state":np.arange(0, 31, 3), "max_iter": [100, 500, 1000]}

# train_test_split 按比例划分原数据集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor,input_lable,train_size=0.8,random_state = 20)

# 建立逻辑回归模型
clf = LR()
# 将模型加入超参数网格搜索和交叉验证     
clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
# 模型训练
clf = clf.fit(Xtrain,Ytrain)
# 模型评估
Y_pred = clf.predict(Xtest)
cm = confusion_matrix(Ytest,Y_pred)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0

print("数据集划分 random_state：" + str(20)  + "时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：20时，
Evaluate：
  accuracy : 0.8333333333333334
  mcc : 0.6139406135149204
  sens : 66.66666666666666
  spec : 91.66666666666666


## LR Feature2 Disorder score

In [16]:
# 设置超参数
param_dict = { "penalty": ["l1","l2"], "C": np.linspace(0.05,1,19), "random_state":np.arange(0, 31, 3), "max_iter": [100, 500, 1000]}

# train_test_split 按比例划分原数据集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature,input_lable1,train_size=0.8,random_state = 0)

# 建立逻辑回归模型
clf = LR()
# 将模型加入超参数网格搜索和交叉验证     
clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
# 模型训练
clf = clf.fit(Xtrain,Ytrain)
# 模型评估
Y_pred = clf.predict(Xtest)
cm = confusion_matrix(Ytest,Y_pred)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0

print("数据集划分 random_state：" + str(0)  + "时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：0时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.6080052052987127
  sens : 60.0
  spec : 95.23809523809523


## KNN Feature1 Amino acid type

In [6]:
# 设置超参数
param_dict = {"n_neighbors": np.arange(1, 11), "weights":["uniform","distance"]}

# 划分训练集和测试集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor,input_lable,train_size=0.8,random_state = 16)
# build model
model = KNeighborsClassifier()
# 将模型加入超参数网格搜索和交叉验证     
model = GridSearchCV(model, param_grid=param_dict, cv=5)
# model train
model = model.fit(Xtrain,Ytrain)
# 模型评估
predict = model.predict(Xtest)
cm = confusion_matrix(Ytest,predict)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(16)  + "时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：16时，
Evaluate：
  accuracy : 0.8888888888888888
  mcc : 0.7591973244147158
  sens : 84.61538461538461
  spec : 91.30434782608695


## KNN Feature2 Disorder score

In [9]:
# 设置超参数  
param_dict = {"n_neighbors": np.arange(1, 11), "weights":["uniform","distance"]}

# 划分训练集和测试集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature,input_lable1,train_size=0.8,random_state = 17)
# build model
model = KNeighborsClassifier()
# 将模型加入超参数网格搜索和交叉验证     
model = GridSearchCV(model, param_grid=param_dict, cv=5)
# model train
model = model.fit(Xtrain,Ytrain)
# 模型评估
predict = model.predict(Xtest)
cm = confusion_matrix(Ytest,predict)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(17)  + "时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：17时，
Evaluate：
  accuracy : 0.8888888888888888
  mcc : 0.7230769230769231
  sens : 80.0
  spec : 92.3076923076923


## SVM Feature1 Amino acid type

In [12]:
# 设置超参数
param_dict = {"C":[10 ** i for i in range(-5, 6)], "kernel": ["linear","poly","rbf","sigmoid"], "gamma": np.logspace(-10, 1, 50),
              "decision_function_shape":["ovo", "ovr"]}

# 划分训练集和测试集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor,input_lable,train_size=0.8,random_state = 0)

clf= SVC(degree = 1
         , cache_size=5000
         , random_state = 30
         )
# 将模型加入超参数网格搜索和交叉验证     
clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
clf = clf.fit(Xtrain,Ytrain)

Y_pred = clf.predict(Xtest)
cm = confusion_matrix(Ytest,Y_pred)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(0)  + "，定义SVM模型 random_state：" + str(30) + "时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：0，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.8333333333333334
  mcc : 0.6549455359345627
  sens : 73.33333333333333
  spec : 90.47619047619048


## SVM Feature2 Disorder score

In [22]:
# 设置超参数
param_dict = {"C":[10 ** i for i in range(-5, 6)], "kernel": ["linear","poly","rbf","sigmoid"], "gamma": np.logspace(-10, 1, 50),
              "decision_function_shape":["ovo", "ovr"]}

# 划分训练集和测试集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature,input_lable1,train_size=0.8,random_state = 13)
clf= SVC(degree = 1
         , cache_size=5000
         , random_state = 30
         )
# 将模型加入超参数网格搜索和交叉验证     
clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
clf = clf.fit(Xtrain,Ytrain)

Y_pred = clf.predict(Xtest)
cm = confusion_matrix(Ytest,Y_pred)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(13)  + "，定义SVM模型 random_state：" + str(30) + "时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：13，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.9166666666666666
  mcc : 0.828302688169126
  sens : 92.85714285714286
  spec : 90.9090909090909


## DT  Feature1   Amino acid type

In [12]:
# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

# 按比例划分原数据集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor, input_lable, train_size=0.8, random_state=16)
# 定义模型  
clf = tree.DecisionTreeClassifier(criterion="entropy")
# 将模型加入超参数网格搜索和交叉验证
clf = GridSearchCV(clf, param_grid=param_dict, cv=5) #将训练/测试数据集划分5个互斥子集
# 训练数据集
clf = clf.fit(Xtrain, Ytrain)   
Y_pred = clf.predict(Xtest)
cm = confusion_matrix(Ytest,Y_pred) # 混淆矩阵
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn


accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(16) + " 时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：16 时，
Evaluate：
  accuracy : 0.8888888888888888
  mcc : 0.7722492140123949
  sens : 92.3076923076923
  spec : 86.95652173913044


## DT Feature2 Disorder score

In [14]:
# 设置超参数  
param_dict = {"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}
# 按比例划分原数据集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature, input_lable1, train_size=0.8, random_state=17)
# 定义模型  
clf = tree.DecisionTreeClassifier(criterion="entropy")
# 将模型加入超参数网格搜索和交叉验证
clf = GridSearchCV(clf, param_grid=param_dict, cv=5) #将训练/测试数据集划分5个互斥子集
# 训练数据集
clf = clf.fit(Xtrain, Ytrain)   
Y_pred = clf.predict(Xtest)
cm = confusion_matrix(Ytest,Y_pred) # 混淆矩阵
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(17) + " 时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：17 时，
Evaluate：
  accuracy : 0.8611111111111112
  mcc : 0.6656822568860216
  sens : 80.0
  spec : 88.46153846153845


## XGBoost Feature1 Amino acid type

In [27]:
# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

# train_test_split 按比例划分原数据集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor,input_lable,train_size=0.8,random_state = 4)
# 定义模型
model = xgb.XGBClassifier()
# 将模型加入超参数网格搜索和交叉验证     
model = GridSearchCV(model, param_grid=param_dict, cv=5)
# 训练数据集
model = model.fit(Xtrain, Ytrain)
# 模型评估
predict = model.predict(Xtest)
cm = confusion_matrix(Ytest,predict)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(4) + " 时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.9166666666666666
  mcc : 0.8101627221513197
  sens : 90.9090909090909
  spec : 92.0


## XGBoost Feature2 Disorder score

In [28]:
# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

# train_test_split 按比例划分原数据集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature,input_lable1,train_size=0.8,random_state = 13)
# 定义模型
model = xgb.XGBClassifier()
# 将模型加入超参数网格搜索和交叉验证     
model = GridSearchCV(model, param_grid=param_dict, cv=5)
# 训练数据集
model = model.fit(Xtrain, Ytrain)
# 模型评估
predict = model.predict(Xtest)
cm = confusion_matrix(Ytest,predict)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(13) + " 时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：13 时，
Evaluate：
  accuracy : 0.9444444444444444
  mcc : 0.8831168831168831
  sens : 92.85714285714286
  spec : 95.45454545454545


## RF Feature1 Amino acid type

In [29]:
# 设置超参数  n_estimators：决策数的数量 默认100
param_dict = {"n_estimators": [100, 200, 400, 700], "max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 5)}

# train_test_split 按比例划分原数据集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor,input_lable,train_size=0.8,random_state = 23)
# 定义模型
model = RandomForestClassifier(criterion="entropy")
# 将模型加入超参数网格搜索和交叉验证     
model = GridSearchCV(model, param_grid=param_dict, cv=5)
# 训练数据集
model = model.fit(Xtrain, Ytrain)
# 模型评估
predict = model.predict(Xtest)
cm = confusion_matrix(Ytest,predict)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(23) + " 时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：23 时，
Evaluate：
  accuracy : 0.8888888888888888
  mcc : 0.7662337662337663
  sens : 85.71428571428571
  spec : 90.9090909090909


## RF Feature2 Disorder score

In [30]:
# 设置超参数  n_estimators：决策数的数量 默认100
param_dict = {"n_estimators": [100, 200, 400, 700], "max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 5)}

# train_test_split 按比例划分原数据集
Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature,input_lable1,train_size=0.8,random_state = 13)
# 定义模型
model = RandomForestClassifier(criterion="entropy")
# 将模型加入超参数网格搜索和交叉验证     
model = GridSearchCV(model, param_grid=param_dict, cv=5)
# 训练数据集
model = model.fit(Xtrain, Ytrain)
# 模型评估
predict = model.predict(Xtest)
cm = confusion_matrix(Ytest,predict)
tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
n = tp + fp + tn + fn
accuracy = (tp + tn)/n 
mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
print("数据集划分 random_state：" + str(13) + " 时，")
print("Evaluate：")
print("  accuracy : " + str(accuracy))
print("  mcc : " + str(mcc))
print("  sens : " + str(sens))
print("  spec : " + str(spec)) 
print("============================")

数据集划分 random_state：13 时，
Evaluate：
  accuracy : 0.9166666666666666
  mcc : 0.828302688169126
  sens : 92.85714285714286
  spec : 90.9090909090909
